In [0]:
df = spark.read.option("recursiveFileLookup", "true").parquet("s3://s3-lge-he-inbound-kic-prd/tlamp/ucm/adhoc/*.parquet")
display(df)

In [0]:
df.count()

In [0]:
from pyspark.sql.functions import sha2, expr, current_timestamp, col, when, concat, lit, date_format, split, size, substring, to_timestamp
from pyspark.sql.types import DoubleType, TimestampType

he_etl_dt = current_timestamp()
tv_salt = dbutils.secrets.get("admin", "salt")

In [0]:
# 컬럼 추가
df = df.withColumn("he_etl_dt", he_etl_dt)
# df = df.withColumn("src_file_path", expr("_metadata.file_path"))

df = df.withColumn("mac_addr", when(col("mac").isNull() | (col("mac") == ''), col("mac")).otherwise(sha2(concat(col("mac"), lit(tv_salt)), 256)))

# mac 칼럼 제거 및 mac_addr을 제일 앞에 배치
cols = [c for c in df.columns if c not in ("mac", "user_number")]
cols = ["mac_addr"] + [c for c in cols if c != "mac_addr"]
df = df.select(cols)

In [0]:
display(df)

In [0]:
df.count()

In [0]:
%sql
select 
  count(distinct mac_addr), count(1)
from adhoc.heds.1256_targer_mac

In [0]:
%sql
select 
  date_ym, count(mac_addr)
from adhoc.heds.1256_targer_mac
group by 
  date_ym

In [0]:
target_mac_df = spark.table("adhoc.heds.1256_targer_mac")
intersect_count = df.select("mac_addr").intersect(target_mac_df.select("mac_addr")).count()
intersect_count

In [0]:
df.write.saveAsTable("adhoc.heds.1282_ucm_mac")

In [0]:
%sql
select count(1), count(distinct mac_addr)
from adhoc.heds.1282_ucm_mac

In [0]:
%sql
select count(1), count(distinct mac_addr)
from adhoc.heds.1282_ucm_mac
where 1=1
  and mac_addr in (select distinct mac_addr from adhoc.heds.1256_targer_mac)

In [0]:
%sql
select 
  nvl(mac_addr, 'NULL_') as mac_addr,
  nvl(product, 'NULL_') as product,
  nvl(country, 'NULL_') as country,
  nvl(channel_code, 'NULL_') as channel_code,
  nvl(source_type, 'NULL_') as source_type,
  nvl(event_start_time, 'NULL_') as event_start_time,
  nvl(event_end_time, 'NULL_') as event_end_time,
  nvl(split_event_start_time, 'NULL_') as split_event_start_time,
  nvl(split_event_end_time, 'NULL_') as split_event_end_time,
  nvl(split_event_start_local_time, 'NULL_') as split_event_start_local_time,
  nvl(split_event_end_local_time, 'NULL_') as split_event_end_local_time,
  nvl(log_time, 'NULL_') as log_time,
  nvl(s3_path_time, 'NULL_') as s3_path_time,
  nvl(content_set_id, 'NULL_') as content_set_id,
  nvl(content_id, 'NULL_') as content_id,
  nvl(program_group_id, 'NULL_') as program_group_id,
  nvl(schedule_id, 'NULL_') as schedule_id,
  nvl(episode_number, 'NULL_') as episode_number,
  nvl(schedule_program_title, 'NULL_') as schedule_program_title,
  nvl(schedule_sub_title, 'NULL_') as schedule_sub_title,
  nvl(channel_name, 'NULL_') as channel_name,
  nvl(genre_image_info, 'NULL_') as genre_image_info,
  nvl(program_image_info, 'NULL_') as program_image_info,
  nvl(lg_genre_name, 'NULL_') as lg_genre_name,
  nvl(lg_genre_code, 'NULL_') as lg_genre_code,
  nvl(lg_genre_name2, 'NULL_') as lg_genre_name2,
  nvl(lg_genre_code2, 'NULL_') as lg_genre_code2,
  nvl(cast_name, 'NULL_') as cast_name,
  nvl(actor_name, 'NULL_') as actor_name,
  nvl(actor_ids, 'NULL_') as actor_ids,
  nvl(director_name, 'NULL_') as director_name,
  nvl(director_ids, 'NULL_') as director_ids,
  nvl(producer_name, 'NULL_') as producer_name,
  nvl(producer_ids, 'NULL_') as producer_ids,
  nvl(keyword_character, 'NULL_') as keyword_character,
  nvl(keyword_subject, 'NULL_') as keyword_subject,
  nvl(keyword_setting, 'NULL_') as keyword_setting,
  nvl(keyword_time_period, 'NULL_') as keyword_time_period,
  nvl(keyword_theme, 'NULL_') as keyword_theme,
  nvl(keyword_mood, 'NULL_') as keyword_mood,
  nvl(keyword_general, 'NULL_') as keyword_general,
  nvl(meta_start_time, 'NULL_') as meta_start_time,
  nvl(meta_end_time, 'NULL_') as meta_end_time,
  nvl(meta_create_date, 'NULL_') as meta_create_date,
  nvl(genre_name, 'NULL_') as genre_name,
  nvl(genre_name2, 'NULL_') as genre_name2,
  nvl(watch_start_time, 'NULL_') as watch_start_time,
  nvl(watch_end_time, 'NULL_') as watch_end_time,
  nvl(schedule_duration, 'NULL_') as schedule_duration,
  nvl(watch_duration, 'NULL_') as watch_duration,
  nvl(watch_ratio, 'NULL_') as watch_ratio,
  nvl(ip_chan_code, 'NULL_') as ip_chan_code,
  nvl(ip_org_chan_code, 'NULL_') as ip_org_chan_code,
  nvl(domain, 'NULL_') as domain,
  nvl(local_watch_start_time, 'NULL_') as local_watch_start_time,
  nvl(local_watch_end_time, 'NULL_') as local_watch_end_time,
  nvl(watch_start_hour, 'NULL_') as watch_start_hour,
  nvl(watch_end_hour, 'NULL_') as watch_end_hour,
  nvl(watch_date, 'NULL_') as watch_date,
  nvl(watch_day, 'NULL_') as watch_day,
  nvl(he_etl_dt, 'NULL_') as he_etl_dt
from adhoc.heds.1282_ucm_mac
where 1=1
  and mac_addr in (select distinct mac_addr from adhoc.heds.1256_targer_mac)

In [0]:
df_filtered = spark.sql("""
select 
  *
from adhoc.heds.1282_ucm_mac
where 1=1
  and mac_addr in (select distinct mac_addr from adhoc.heds.1256_targer_mac)
""")
df_filtered.coalesce(1).write.mode("overwrite").parquet("/Volumes/sandbox/z_yeswook_kim/v_yeswook_kim/parquet/1282")

In [0]:
df.write.saveAsTable("sandbox.z_jihyeon55_oh.1282_ucm_mac")

In [0]:
    
target_mac_df.write.saveAsTable("sandbox.z_jihyeon55_oh.1256_target_mac")